### IO
这一节详细分析下Langchain的IO部分，也是最基础的部分。一般来说无论你的应用多么复杂或者是多么简单，提示词，语言模型，输出控制这三部分都是有的。而这三部分刚好构成一个对LLM的输入和输出。我们先来说说LLM这部分。

#### LLM和ChatLLM
在当前时间点，市场上已经出现了各种各样大量的语言模型，我们上次分享也提到了一些其中比较出名和优秀的。对于这些模型的调用方式一般来说可以分为三种情况：第一类就是本质是以服务的方式提供的，无论是否开源，比如ChatGPT，比如通义千问。这类服务一般同时会提供SDK，让你通过它提供的sdk调用比如openai的包就可以调用ChatGPT等模型。第二类情况就是自部署的，比如千问大模型，这类模型一般来说都会以restful方式提供访问，或者通过提供和某个模型（比如ChatGPT）一样格式的restful接口，使得你可以直接使用openai的包，这个我们上次分享也以千问大模型举过例子。最后一种就是通过平台调用的方式去使用模型，比如Huggingface或者Ollama。这种一般来说平台会提供一个统一的调用的方式，让你能快速在平台支持的模型之间切换。

Langchain抽象了这些各种模型的调用的底层，实现了它的统一抽象：Runnable接口，这个我们后面会细说。但是从调用层面来说，不同模型在使用的时候其实依然差别不小，甚至使用的包都不一样，比如你用Langchain调用ChatGPT需要使用langchiain_openai这个包，而如果你使用ollama，只需要使用langchain_community包。这种混乱可能也是当前LLM开发高速发展的无奈之举吧，所以如果你要在Langchain中使用一个你之前没有使用过的模型，还是需要查阅[文档](https://python.langchain.com/v0.1/docs/integrations/llms/)。好在虽然混乱，但是Langchain基本囊括了市面上大部分的LLM。你基本上都能在文档中找到调用的方法。加上Chain的思想让应用各个组件之间实现了解耦和隔离，可以方便地快速替换。

<center>
  <img src="images/LLMs.png">
</center>

接下来我们要区分LLM和ChatLLM。其实他们的区别就是LLM的输入一般就是一个字符串，输出也是一个字符串，你可以把它看做一个输入是字符串输出是字符串的黑盒函数。而ChatLLM的输入和输出都是Message列表。这些不同的Message组合层一个列表可以表示对话历史，可以区分角色，也就是可以让用户和模型进行连续对话。一般来说如果要完成一个复杂的应用都需要连续和LLM对话，因此在这次分享中，除了这个notebook之外的其他默认使用的都是ChatLLM模型。

从上面可以看出来，LLM和ChatLLM只是调用方式上的区别，模型依然是同一个模型。不同调用方式的输入和输出也不一样，我们下面以Ollama作为演示。经过上次分享，相信大家电脑上应该都装了Ollama且去折腾了一番了吧。在你学习LLM开发的时候，使用本地模型，可以节约很多钱，因为很多LLM应用涉及到频繁和模型交互，消耗的token数量可以是很夸张的。当然在后面的分享中，我们也会在RAG和Agent的最终例子里通过将Ollama模型替换为通义千问和ChatGPT来观察效果有何不同。

In [1]:
# OllamaCall.ts
from langchain_community.llms import Ollama

llm = Ollama(model='qwen:14b')
llm.invoke("介绍下你自己")

'我是来自阿里云的大规模语言模型，我叫通义千问。我的目标是帮助用户获得准确、有用的信息，解决他们的问题和困惑。我会不断学习和进步，为用户提供更好的服务。请随时告诉我你需要什么帮助。'

In [8]:
# ChatOllamaCall.ts
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model='qwen:14b')
messages = [
    SystemMessage(content='你好，我叫周代琳，我是你的主人，你是一个智能助手，你叫小冰'),
    AIMessage(content='好的，主人'),
    HumanMessage(content='我叫什么，你是谁')
]
llm.invoke(messages)

AIMessage(content='您是周代琳，我的名字是小冰。我是您的智能助手。有什么我可以帮到您的吗？', response_metadata={'model': 'qwen:14b', 'created_at': '2024-06-18T16:13:04.4431162Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 7292237600, 'load_duration': 6602519100, 'prompt_eval_count': 51, 'prompt_eval_duration': 134874000, 'eval_count': 25, 'eval_duration': 552622000}, id='run-ce27b4a7-dbed-4b11-a808-c92db2433945-0')

#### Prompt
提示词应该大家都知道，甚至提示词工程大家也有所耳闻。大家也可能听过一些提示词的手段Few-Shot，CoT，ToT等。我们先来补充一些和提示词相关的知识，然后再来看怎么在Langchain中构造提示词。